# function calling
Function Calling은 AI 모델이 실시간 데이터 조회나 외부 서비스 조작을 위해 필요한 '도구'를 스스로 선택하고 실행 인자를 만들어주는 기술이다.   
모델이 함수를 직접 실행하는 대신 실행에 필요한 데이터를 구조화된 JSON 형태로 짜주면, 개발자의 코드가 이를 처리해 최신 정보를 답변에 반영할 수 있다.

### 공식문서 예시
https://platform.openai.com/docs/guides/function-calling#function-tool-example

In [8]:
import os
import requests
import json
from dotenv import load_dotenv
from pprint import pprint

from openai import OpenAI

load_dotenv()

TMDB_API_KEY = os.getenv('TMDB_API_KEY')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key=OPENAI_API_KEY)

model = 'gpt-4o-mini'


In [5]:

# 1. Define a list of callable tools for the model
tools = [
    {
        "type": "function",
        "name": "get_horoscope",
        "description": "Get today's horoscope for an astrological sign.",
        "parameters": {
            "type": "object",
            "properties": {
                "sign": {
                    "type": "string",
                    "description": "An astrological sign like Taurus or Aquarius",
                },
            },
            "required": ["sign"],
        },
    },
]

def get_horoscope(sign):
    # api 요청이 들어갈 예정입니다.
    return f"{sign}: Next Tuesday you will befriend a baby otter."

# Create a running input list we will add to over time
input_list = [
    {"role": "user", "content": "What is my horoscope? I am an Aquarius."}
]

# 2. Prompt the model with tools defined
response = client.responses.create(
    model=model,
    tools=tools,
    input=input_list,
)

# Save function call outputs for subsequent requests
input_list += response.output

for item in response.output:
    # pprint(item)
    if item.type == "function_call":
        if item.name == "get_horoscope":
            # 3. Execute the function logic for get_horoscope
            horoscope = get_horoscope(json.loads(item.arguments))
            
            # 4. Provide function call results to the model
            input_list.append({
                "type": "function_call_output",
                "call_id": item.call_id,
                "output": json.dumps({
                  "horoscope": horoscope
                })
            })

print("Final input:")

response = client.responses.create(
    model=model,
    instructions="Respond only with a horoscope generated by a tool.",
    tools=tools,
    input=input_list,
)

# 5. The model should be able to give a response!
print("Final output:")
# print(response.model_dump_json(indent=2))
print("\n" + response.output_text)

Final input:
Final output:

Your horoscope for Aquarius: Next Tuesday you will befriend a baby otter.


In [4]:
response = client.responses.create(
    model=model,
    input='안녕',
)

pprint(response.output)

[ResponseOutputMessage(id='msg_07bd74616f2ba8fc00697174d89938819faaf40f1e3602ec33', content=[ResponseOutputText(annotations=[], text='안녕하세요! 어떻게 도와드릴까요?', type='output_text', logprobs=[])], role='assistant', status='completed', type='message')]


### 함수 정의
LLM이 호출할 함수를 정의한다.

In [24]:
def get_now_playing_movies(region):
    """TMDB API를 호출하여 현재 상영 중인 영화 목록을 가져옵니다."""

    print(f'{region}의 영화를 가져옵니다.')

    url = "https://api.themoviedb.org/3/movie/now_playing"
    
    params = {
        'language' : 'ko-kr',
        'region' : region
    }
    
    headers = {
        'Authorization' : f'Bearer {TMDB_API_KEY}'
    }
    
    try:
        response = requests.get(url, params=params, headers=headers)
        response.raise_for_status()    
        return response.json()['results']
    except Exception as e:
        print(e)

# pprint(get_now_playing_movies())

### 함수 스키마 정의
LLM이 함수를 이해할 수 있도록 스키마를 정의한다.

In [10]:
tools = [
    {
        "type": "function",
        "name": "get_now_playing_movies",
        "description": "현재 극장에서 상영 중인 영화 목록을 TMDB에서 가져옵니다.",
        "parameters": {
            "type": "object",
            "properties": {
                "region": {
                    "type": "string",
                    "description": "region, ISO-3166-1 code",
                },

            }
        }
    }
]

In [26]:

def run_conversation(user_prompt):
    
    input_list = [{"role": "user", "content": user_prompt}]

    # 1. LLM에게 1차 요청을 한다.
    response = client.responses.create(
        model=model,
        input=input_list,
        tools=tools,
    )
    input_list += response.output
    
    # 2. LLM의 응답에서 "함수 실행"에 대한 응답이 있다면 이를 실행한다.
    for item in response.output:
        if item.type == "function_call":
            if item.name == 'get_now_playing_movies':

                result = get_now_playing_movies(**json.loads(item.arguments))
                
                # 3. 함수 실행 결과를 input에 담는다.
                input_list.append({
                    "type": "function_call_output",
                    "call_id": item.call_id,
                    "output": json.dumps({
                        "result": result
                        })
                })
    # 4. 다시 한번 LLM에게 요청한다.
    response = client.responses.create(
        model=model,
        instructions="""
        Respond only with a movies generated by a tool. 
        영화 제목만 bullet point 형태로 나열해줘.
        만약 적합한 region이 없으면 응답하지 못한다고 말해줘
        """,

        tools=tools,
        input=input_list,
    )

    return response.output_text
    

# 실행 예시
print(run_conversation("요즘 미국에서 볼만한 영화가 뭐가 있어?"))

US의 영화를 가져옵니다.
- 아바타: 불과 재
- 하우스메이드
- 살레줄
- 28년 후: 뼈 신전
- 그린란드 2
- 마르티 슈프림
- 스폰지밥 무비: 스퀘어팬츠를 찾아서
- 아나콘다
- 얼마나 사랑하고 있지?
- 프라이메이트
- 삶이란 모래사장
- 다비드
- 리턴 투 사일런트 힐
- 그들이 지배하는 날
- 미메스
- 마다가스카르
- 아리엘과의 첫 만남
- 원스 어폰 어 타임
- 셀리마 
- 다크 그림자


In [13]:
# 사용자가 요청합니다.
query = input()
response = client.responses.create(
    model=model,
    input=query,
)
print(response.output_text)

현재 한국에서 상영 중인 영화는 시간이 흘러 변경될 수 있으므로, 최신 정보를 확인하는 것이 가장 좋습니다. 보통 주요 영화관 웹사이트나 영화 예매 사이트(예: CGV, 롯데시네마, 메가박스 등)를 통해 최신 상영작 정보를 찾을 수 있습니다. 최근 개봉한 인기 영화나 화제작 등을 확인해 보세요. 어떤 특정 영화에 대해 궁금하신 점이 있으면 말씀해 주세요!
